In [4]:
import docx
import os
from openpyxl import Workbook
import re  # Import the regular expression module
import pandas as pd
import numpy as np
import math
import warnings
from scipy.stats import norm 

warnings.filterwarnings('ignore')

In [5]:
docx_dir = "C:/Users/anaho/Desktop/research/Ayahuasca/DATA/FinalCodings"

## 1. Get ayahuasca users data


In [6]:
# 1. Get all DOCX files in the directory, sorted alphanumerically
docx_files = sorted([f for f in os.listdir(docx_dir) if f.endswith(".docx")])

all_data = {}  # Store data from all files, keyed by participant ID
df = pd.DataFrame()

#make question dict
question_dict = {}

In [7]:
 # 2. Iterate through each DOCX file
for docx_file in docx_files:
    docx_path = os.path.join(docx_dir, docx_file)
    try:
        document = docx.Document(docx_path)
        num_tables = len(document.tables)
        print(f"Processing {num_tables} tables in {docx_file}")
        if num_tables == 0:
            print(f"No tables found in {docx_file}, skipping.")
            continue

    except docx.exceptions.EmptyPackageError as e:
        print(f"Error reading {docx_file}: {e}.  Skipping this file.")
        continue

    # Extract participant name from filename (remove ".docx")
    participant_name = os.path.splitext(docx_file)[0]  # e.g., "codage_01"

    # Extract participant ID (e.g., "01")
    participant_id = participant_name[-2:]
    
    all_table_data= [] #instantiate list

    # 3. Iterate through each table in the document
    for table_index, table in enumerate(document.tables):
        print(f"  Processing table {table_index + 1} of {num_tables} in {docx_file}")
        table_data = {}  # Store data for the current table, keyed by question code
        # 4. Extract data from the table, handling LetterNumber rows
        for row in table.rows:
            cells = [cell.text.strip() for cell in row.cells]  # Remove extra whitespace
            if cells:
                first_cell_value = cells[0]
                if re.match(r"^[A-Za-z]\d+$", first_cell_value) or re.match(r"^[A-Za-z]\d[A-Za-z]+$", first_cell_value) or re.match(r"^[A-Za-z]\d\d[A-Za-z]+$", first_cell_value):
                    question_code = first_cell_value
                    question_text = cells[1]  # Extract the question text
                    yes_val = cells[2].lower()
                    no_val = cells[3].lower()
                    note = cells[4]
                    response = None # Initialize response

                    if "x" in yes_val:
                        response = 1.0
                    elif "x" in no_val:
                        response = 0.0
                    else:
                         response = 0.0 #replacing by 0 the missing x

                    table_data[question_code] = {"response": response, "note": note}
                    question_dict[question_code] = question_text #store the question and text.

        
            '''    else:
                    print(
                        f"    Skipping row in {docx_file}, table {table_index + 1} because the first cell '{first_cell_value}' does not match 'A1' format.")
            else:
                print(f"    Skipping empty row in {docx_file}, table {table_index + 1}.")'''

        # 5. Store data for this participant and question
        all_table_data.append(table_data)  # Use participant_id

    # Flatten the list of dictionaries into a single dictionary for the participant
    all_table_data_flat = {}
    for item in all_table_data:
        all_table_data_flat.update(item)

    all_data[participant_id] = all_table_data_flat

Processing 3 tables in New_Coding_01.docx
  Processing table 1 of 3 in New_Coding_01.docx
  Processing table 2 of 3 in New_Coding_01.docx
  Processing table 3 of 3 in New_Coding_01.docx
Processing 3 tables in New_Coding_02.docx
  Processing table 1 of 3 in New_Coding_02.docx
  Processing table 2 of 3 in New_Coding_02.docx
  Processing table 3 of 3 in New_Coding_02.docx
Processing 3 tables in New_Coding_04.docx
  Processing table 1 of 3 in New_Coding_04.docx
  Processing table 2 of 3 in New_Coding_04.docx
  Processing table 3 of 3 in New_Coding_04.docx
Processing 3 tables in New_Coding_05.docx
  Processing table 1 of 3 in New_Coding_05.docx
  Processing table 2 of 3 in New_Coding_05.docx
  Processing table 3 of 3 in New_Coding_05.docx
Processing 3 tables in New_Coding_06.docx
  Processing table 1 of 3 in New_Coding_06.docx
  Processing table 2 of 3 in New_Coding_06.docx
  Processing table 3 of 3 in New_Coding_06.docx
Processing 3 tables in New_Coding_07.docx
  Processing table 1 of 3 in

In [8]:
 # 6. Prepare data for Excel, transposing and combining
# Prepare data for Excel sheet
excel_data = []
# Get all unique questions
unique_questions = set()
for data in all_data.values():
    unique_questions.update(data.keys())
unique_questions = sorted(list(unique_questions))

# Create the header row
header_row = ["Participants"]  # Changed Header
for question in unique_questions:
    header_row.extend([f"{question}"])
excel_data.append(header_row)

# Create participant rows
participant_ids = sorted(list(all_data.keys()))  # Get unique participant IDs
for participant_id in participant_ids:
    participant_data = all_data[participant_id]
    participant_row = [participant_id]  # Add participant ID here
    for question in unique_questions:
        if question in participant_data:
            #participant_row.extend([participant_data[question]['response'], participant_data[question]['note']]) #including notes
            participant_row.extend([participant_data[question]['response']])  #only codes, no notes
        if question not in participant_data:
            print(question)
    excel_data.append(participant_row)

# 7. Put everything in a dataframe!!
df = pd.DataFrame(excel_data, columns = header_row).drop([0])
df_final = df.transpose().drop(['Participants'])

# Convert all columns to numeric, coercing errors to NaN
for col in df_final.columns:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

In [9]:
#keep biggest value for outcome and interpreration
homo_codes = {'H1':'P1','H2':'P2','H3':'P3','H4':'P4','H5':'P5','H6':'P6','H7':'P7','H8':'P8',
              'Q1':'I1','Q2':'I2','Q3':'I3', 'Q4':'I4a','Q5':'I5','Q6':'I6'} #homologous codes


counter = 1 # Initialize a counter for your T1, T2, T3... row names

for h_code, p_i_code in homo_codes.items():
    # Check if both rows exist in the DataFrame to prevent errors
    if h_code in df_final.index and p_i_code in df_final.index:
        # Generate the new row name (e.g., 'T1', 'T2')
        new_row_name = f'T{counter}'
        question_dict[f'T{counter}'] = 'overall ' + question_dict[h_code]

        # Perform the comparison and store the biggest value in the new row
        df_final.loc[new_row_name] = np.maximum(df_final.loc[h_code], df_final.loc[p_i_code])

        print(f"Created row '{new_row_name}' by comparing '{h_code}' and '{p_i_code}'.")
        counter += 1 # Increment the counter for the next row
    else:
        print(f"Warning: Skipping comparison for '{h_code}' and '{p_i_code}'. One or both indices not found in DataFrame.")


df_final

Created row 'T1' by comparing 'H1' and 'P1'.
Created row 'T2' by comparing 'H2' and 'P2'.
Created row 'T3' by comparing 'H3' and 'P3'.
Created row 'T4' by comparing 'H4' and 'P4'.
Created row 'T5' by comparing 'H5' and 'P5'.
Created row 'T6' by comparing 'H6' and 'P6'.
Created row 'T7' by comparing 'H7' and 'P7'.
Created row 'T8' by comparing 'H8' and 'P8'.
Created row 'T9' by comparing 'Q1' and 'I1'.
Created row 'T10' by comparing 'Q2' and 'I2'.
Created row 'T11' by comparing 'Q3' and 'I3'.
Created row 'T12' by comparing 'Q4' and 'I4a'.
Created row 'T13' by comparing 'Q5' and 'I5'.
Created row 'T14' by comparing 'Q6' and 'I6'.


,1,2,3,4,5,6,7,8,9,10,11,12,13,14
A1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
A2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
A4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T10,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T11,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
T12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
T13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_final['Total'] = df_final.astype(float).sum(axis = 1)

#add col containing textual question
df_final['Question_text'] = df_final.index.map(question_dict)

#get percentages
df_final['Percentages'] = np.round(df_final['Total']/14*100,2) #dep on the nb of part

df_final.loc['J1':'O9','Percentages'] = np.round(df_final.loc['J1':'O9','Total']/14*100,2) #only 14 reported a form of VH after


In [11]:
df_final.to_csv(docx_dir + 'descriptive_stats.csv', index=True)

In [12]:
#very low scores
list(df_final[df_final['Total']<2].Question_text)

['First experiences connected to use of psychedelics or other drugs?',
 'Volitional',
 'Non-volitional',
 'Ability to influence voice',
 'Change in influence over time',
 'Change in frequency over time',
 'Voice character change over time',
 'Olfactory hallucination',
 'Gustatory hallucination',
 'Dissociative experiences',
 'Non-verbal sounds',
 'Visual hallucination',
 'Tactile hallucination',
 '‘Boundary’ voices',
 '‘Egocentric voices',
 '‘Allocentric’ voices',
 'Depression-associated',
 'Paranoia-associated',
 'Absent agency',
 'Agency without individuation',
 'Internally individualised agency',
 'Calm, relaxed state',
 'Anxious, stressed or alert state',
 'Depressed state',
 'Events that recently happened',
 'Absence of surrounding people',
 'Frequency',
 'Intensity',
 'Decreasing sense of control over time',
 'Negative impact on relationships',
 'Family narrative',
 'Self-stigma regarding voices',
 '‘Auditory’ voices',
 'Olfactory hallucination',
 'Gustatory hallucination',
 'Dis

In [13]:
df_final[df_final['Question_text'] == 'Spiritual beliefs preceded first VH experience']

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total,Question_text,Percentages
I4b,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,8.0,Spiritual beliefs preceded first VH experience,57.14


In [14]:
#more than majority
list(df_final[df_final['Total']>=8].Question_text)

['Bodily states',
 '‘Thought-like’ voices',
 'Multisensory voices',
 'Visual imagery',
 'Internally located voices?',
 'Recurring voices',
 'Direct address',
 'Commenting voices?',
 'Conversational voices?',
 'Commanding voices?',
 'Follow commands?',
 'Positive/helpful voices?',
 'Contains a message',
 'Recognisable voices',
 'Voice knowledge',
 'Complex personification',
 'Archetypal features',
 'Voices elicit positive emotions',
 'Voices elicit negative emotions',
 'Externally individualised agency',
 'Presence of voices',
 'State of mind',
 'Presence or actions of facilitators',
 'Music',
 'Impact of voices on quality of life',
 'Pedagogical outcome',
 'Therapeutic outcome',
 'Spiritual/religious outcome',
 'Supernatural/spiritual narrative',
 'Spiritual beliefs preceded first VH experience',
 'Bodily states',
 '‘Thought-like’ voices',
 'Multisensory voices',
 'Visual imagery',
 'Felt presences',
 'Internally located voices?',
 'Recurring voices',
 'Direct address',
 'Commenting vo

In [15]:
#very high scores
list(df_final[df_final['Total']>=13].Question_text)

['‘Thought-like’ voices',
 'Internally located voices?',
 'Recurring voices',
 'Positive/helpful voices?',
 'Contains a message',
 'Voice knowledge',
 'Voices elicit positive emotions',
 'Externally individualised agency',
 '‘Thought-like’ voices',
 'Positive/helpful voices?',
 'Contains a message',
 'Recognisable voices',
 'Voices elicit positive emotions',
 'Impact of voices on quality of life',
 'Supernatural/spiritual narrative',
 'overall Impact of voices on quality of life',
 'overall Supernatural/spiritual narrative']

 ## Comparing with other groups from Moseley et al. 2023

### Data from Mosely et al., 2023  paper

In [16]:
data_table1 = np.array([['Visual imagery', 100, 5.0],
                 ['Auditory', 84.6, 92.5],
                 ['Internally located', 84.6, 62.5],
                 ['Egocentric', 80.8, 70.0],
                 ['Thought-like', 76.9, 52.5],
                 ['Externally located', 73.1, 80.0],
                 ['Felt presence', 69.2, 52.5],
                 ['Multimodal', 69.2, 27.5],
                 ['Visual', 60.0, 75.0],
                 ['Bodily states', 53.8, 65.0],
                 ['Olfactory', 50.0, 37.5],
                 ['Dissociative', 46.2, 30.0],
                 ['Tactile', 46.2, 22.5],
                 ['Gustatory', 26.9, 0],
                 ['Nonverbal', 26.9, 40.0],
                 ['Allocentric', 23.1, 37.5],
                 ['Boundary', 11.5, 35.0],
                 ['Elicit positive emotions', 100 ,35]])
                 
data_table2= np.array([['Voice knowledge', 96.2, 45],
                 ['Volitional', 92.3, 2.5],
                 ['Nonvolitional', 92.3, 100],
                 ['Positive/helpful', 92.3, 42.5],
                 ['Recurring', 92.3, 92.5],
                 ['Ability to influence', 84.6, 27.5],
                 ['Change in frequency', 80.8, 70],
                 ['Recognizable', 76.9, 47.5],
                 ['Change in influence', 73.1, 12.5],
                 ['Simple structure', 69.2, 40 ],    
                 ['Conversational', 65.4, 47.5 ],     
                 ['Direct address', 61.5, 82.5 ],     
                 ['Structural change', 50, 80],     
                 ['First voice traumatic event', 42.3, 65],   
                 ['Companionship', 42.3, 32.5],     
                 ['Elicit negative emotions', 38.5, 100 ],  
                 ['Commanding', 26.9, 67.5],
                 ['Commenting', 19.2, 45  ],   
                 ['Character change', 11.5, 17.5],    
                 ['Abusive/violent', 11.5, 87.5]])    
                      
data_table3 =np.array([['Supernatural narrative', 100, 20],
                  ['Internally individualized', 100, 75],    
                  ['Externally individualized', 80.8, 50],      
                  ['Minimal personification', 61.5, 60],      
                  ['Important to identity', 57.7, 12.5],      
                  ['Complex personification', 38.5, 40],     
                  ['Agency w/o individuation', 38.5, 45],
                  ['Self-stigma', 30.8, 47.5],      
                  ['Negative on relationships', 26.9, 77.5],      
                  ['Biophysical narrative', 23.1, 25  ],     
                  ['Archetypal', 23.1, 42.5],     
                  ['Family narrative', 19.2, 22.5],      
                  ['Positive on relationships', 15.4, 5],   
                  ['Trauma narrative', 15.4, 25],    
                  ['Sleep disruption', 15.4, 62.5],      
                  ['Idiosyncratic narrative', 11.5, 32.5],      
                  ['Stress narrative', 11.5, 37.5],      
                  ['Absent agency', 11.5, 15],    
                  ['Suicidality', 3.8, 50]])      
                        

In [17]:
#creating dictionnaries to find codes reported in Moseley paper in our data
#during ayahuasca
dict_during = {  'Visual imagery': 'B7',
                 'Auditory': 'B1',
                 'Internally located': 'C1',
                 'Egocentric': 'C4',
                 'Thought-like': 'B2',
                 'Externally located': 'C2',
                 'Felt presence': 'B8',
                 'Multimodal': 'B4',
                 'Visual': 'B6',
                 'Bodily states': 'B13',
                 'Olfactory': 'B10',
                 'Dissociative': 'B12',
                 'Tactile': 'B9',
                 'Gustatory': 'B11',
                 'Nonverbal': 'B5',
                 'Allocentric': 'C5',
                 'Boundary': 'C3',
                 'Elicit positive emotions': 'D7',
                 'Voice knowledge': 'D20',
                 'Volitional': 'G2',
                 'Nonvolitional': None ,
                 'Positive/helpful': 'D17',
                 'Recurring' : 'D1',
                 'Ability to influence': 'X1', #if any control (g) is 1
                 'Change in frequency': 'D21',
                 'Recognizable': 'D2',
                 'Change in influence' : 'G9', #unsure
                 'Simple structure': 'D10',
                 'Conversational': 'D13',
                 'Direct address': 'D11',
                 'Structural change': None,
                 'First voice traumatic event' : None,
                 'Companionship': 'D18',
                 'Elicit negative emotions': 'D8',
                 'Commanding': 'D14',
                 'Commenting': 'D12',
                 'Character change': 'D6',
                 'Abusive/violent': 'D16',
                 'Absent agency': 'E1',
                 'Internally individualized': 'E3',
                 'Externally individualized': 'E4',
                 'Minimal personification': 'D3',
                 'Important to identity': None,
                 'Complex personification': 'D4',
                 'Archetypal' : 'D5',
                 'Agency w/o individuation': 'E2'}

dict_after = {  'Visual imagery': 'J7',
                 'Auditory': 'J1',
                 'Internally located': 'K1',
                 'Egocentric': 'K4',
                 'Thought-like': 'J2',
                 'Externally located': 'K2',
                 'Felt presence': 'J8',
                 'Multimodal': 'J4',
                 'Visual': 'J6',
                 'Bodily states': 'J13',
                 'Olfactory': 'J10',
                 'Dissociative': 'J12',
                 'Tactile': 'J9',
                 'Gustatory': 'J11',
                 'Nonverbal': 'J5',
                 'Allocentric': 'K5',
                 'Boundary': 'K3',
                 'Elicit positive emotions': 'L7',
                 'Voice knowledge': 'L20',
                 'Volitional': 'O2',
                 'Nonvolitional': None ,
                 'Positive/helpful': 'L17',
                 'Recurring' : 'L1',
                 'Ability to influence': 'X2', #if any control (O) is 1
                 'Change in frequency': 'L21',
                 'Recognizable': 'L2',
                 'Change in influence' : 'O9', #unsure
                 'Simple structure': 'L10',
                 'Conversational': 'L13',
                 'Direct address': 'L11',
                 'Structural change': None,
                 'First voice traumatic event' : None,
                 'Companionship': 'L18',
                 'Elicit negative emotions': 'L8',
                 'Commanding': 'L14',
                 'Commenting': 'L12',
                 'Character change': 'L6',
                 'Abusive/violent': 'L16',
                 'Absent agency': 'M1',
                 'Internally individualized': 'M3',
                 'Externally individualized': 'M4',
                 'Minimal personification': 'L3',
                 'Important to identity': None,
                 'Complex personification': 'L4',
                 'Archetypal' : 'L5',
                 'Agency w/o individuation': 'M2'}

dict_both = {'Supernatural narrative': 'T12',
             'Self-stigma': 'T14',
             'Negative on relationships' : 'T3' ,
             'Biophysical narrative': 'T9',
             'Family narrative': 'T13',
             'Positive on relationships': 'T2',
             'Trauma narrative': None,
             'Sleep disruption': None,
             'Idiosyncratic narrative': 'T11',
             'Stress narrative': None,
             'Suicidality': None}

#number of participants for confidence intervals calculations
N_VIP = 40
N_SPIRI = 26
N_AYA = 15

In [18]:
N=14
#during
df_final.loc['X1',:N]  = df_final.loc[['G1','G2','G3','G4', 'G5', 'G6', 'G7', 'G8'], :N].sum(axis=0) > 0
df_final.loc['X1', :N] = df_final.loc['X1', :N].astype(int)
df_final.loc['X1','Question_text'] = 'Ability_to_control_during'
df_final.loc['X1', 'Total'] = df_final.loc['X1', :N].sum()
df_final.loc['X1', 'Percentages'] = df_final.loc['X1','Total']/N*100

#sam for after
df_final.loc['X2',:N]  = df_final.loc[['O1','O2','O3','O4', 'O5', 'O6', 'O7', 'O8'], :N].sum(axis=0) > 0
df_final.loc['X2', :N] = df_final.loc['X2', :N].astype(int)
df_final.loc['X2','Question_text'] = 'Ability_to_control_during'
df_final.loc['X2', 'Total'] = df_final.loc['X2', :N].sum()
df_final.loc['X2', 'Percentages'] = df_final.loc['X2','Total']/N*100

In [19]:
df_final[df_final['Question_text'] == 'Spiritual beliefs preceded first VH experience']
#df_final.loc[['X1','X2']]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Total,Question_text,Percentages
I4b,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,8.0,Spiritual beliefs preceded first VH experience,57.14


### Adding our data

In [20]:
#instantiating the dataframe
data_tables = np.vstack([data_table1, data_table2, data_table3])
df_others = pd.DataFrame(data_tables, columns=['Code', 'Spiritual%', 'Psychosis%'])

# Set the first element of each row as the index
df_others.index = df_others['Code']
df_others = df_others.drop('Code', axis = 1)


In [21]:
#force it to float
# Convert all columns to numeric, coercing errors to NaN
for col in df_others.columns:
    df_others[col] = pd.to_numeric(df_others[col], errors='coerce')

In [22]:
# populate the comparative DF with our percentages
#category: both
df_both = df_others
for other_code, aya_code in dict_both.items():
    if aya_code is not None:
        df_both.loc[other_code,'Ayahuasca%'] = (df_final.loc[aya_code, 'Percentages'])
df_both = df_both.dropna()
        
#category: during
df_during = df_others
for other_code, aya_code in dict_during.items():
    if aya_code is not None:
        df_during.loc[other_code,'Ayahuasca%'] = (df_final.loc[aya_code, 'Percentages'])
df_during = df_during.dropna()
        
#category: after
df_after = df_others
for other_code, aya_code in dict_after.items():
    if aya_code is not None:
        df_after.loc[other_code,'Ayahuasca%'] = (df_final.loc[aya_code, 'Percentages'])
df_after = df_after.dropna()

### Comparing

In [23]:
# Function to calculate LOR, its 95% CI, and P-value
def calculate_lor_and_ci(row, col_1, col_2, N_SPIRI = N_SPIRI, N_VIP= N_VIP): # Added N_SPIRI and N_VIP as arguments for clarity
    # Define the output column names for returning NaNs
    output_cols = ['LOR', 'LOR_CI_Lower', 'LOR_CI_Upper', 'P_Value'] # Added P_Value

    p1 = row[col_1] / 100
    p2 = row[col_2] / 100
    n1 = N_SPIRI
    n2 = N_VIP

    #account for missing ayahuasca data
    if any(pd.isna(row[col]) for col in [col_1,col_2]):
        print(f"Skipping row due to NaN in required columns: {row.name}") # Optional: for debugging
        return pd.Series([np.nan] * len(output_cols), index=output_cols)

    # Convert proportions to counts (for the 2x2 table)
    # Using 'success' for the percentage value, 'failure' for the rest
    a = round(p1 * n1)
    b = n1 - a
    c = round(p2 * n2)
    d = n2 - c

    # Apply Haldane's correction for zero counts
    # This is crucial for valid SE calculation and avoids inf/-inf
    a_corr = a + 0.5
    b_corr = b + 0.5
    c_corr = c + 0.5
    d_corr = d + 0.5

    # Calculate Log Odds Ratio (using natural log)
    try:
        lor = math.log((a_corr * d_corr) / (b_corr * c_corr))
    except ValueError: # This handles cases where any value is <= 0 after correction (shouldn't happen with 0.5)
        lor = np.nan
        se_lor = np.nan
        lor_ci_lower = np.nan
        lor_ci_upper = np.nan
        or_ci_lower = np.nan
        or_ci_upper = np.nan
        p_value = np.nan # Set p_value to NaN as well
        return pd.Series([lor, lor_ci_lower, lor_ci_upper, p_value], index=output_cols) # Return with p_value


    # Calculate Standard Error of Log Odds Ratio
    se_lor = math.sqrt(1/a_corr + 1/b_corr + 1/c_corr + 1/d_corr)

    # Z-value for 95% CI (two-tailed)
    z_value = 1.96

    # Calculate LOR Confidence Interval
    lor_ci_lower = lor - z_value * se_lor
    lor_ci_upper = lor + z_value * se_lor

    # Transform back to Odds Ratio Confidence Interval (not directly returned by your function but good to keep)
    or_ci_lower = math.exp(lor_ci_lower)
    or_ci_upper = math.exp(lor_ci_upper)

    # *** Calculate the P-value ***
    if se_lor == 0: # Avoid division by zero if SE is somehow 0 (unlikely with Haldane's but good practice)
        p_value = np.nan
    else:
        z_statistic = lor / se_lor
        # For a two-tailed test, we take 2 * (1 - CDF(abs(Z-statistic)))
        p_value = 2 * (1 - norm.cdf(abs(z_statistic)))


    return pd.Series([lor, np.round(lor_ci_lower, 3), np.round(lor_ci_upper, 3), np.round(p_value*3, 4)], # Rounded p_value, p_value x 2 to correct for multiple comparisons
                     index=output_cols)


In [43]:
import pandas as pd
import numpy as np

def df_to_complex_lor_latex_table(df: pd.DataFrame, caption: str = "Logistic Odds Ratios and 95% Confidence Intervals"):
    """
    Converts a pandas DataFrame into a specific LaTeX table format for LORs.

    The input DataFrame should have the following columns:
    - 'Category': Optional string, e.g., 'Impact', 'Narrative'. Used for bolded headers.
    - 'Item': String, the descriptive item name for each row.
    - 'Spiritual%': Float, percentage for Spiritual group.
    - 'Psychosis%': Float, percentage for Psychosis group.
    - 'Ayahuasca%': Float, percentage for Ayahuasca group.
    - '1_LOR', '1_LOR_CI_Lower', '1_LOR_CI_Upper', '1_p_val': Data for comparison 1.
    - '2_LOR', '2_LOR_CI_Lower', '2_LOR_CI_Upper', '2_p_val': Data for comparison 2.
    - '3_LOR', '3_LOR_CI_Lower', '3_LOR_CI_Upper', '3_p_val': Data for comparison 3.

    NaN values in the DataFrame will be represented by a hyphen '-'.
    P-values less than 0.05 will add an asterisk (*) to the CI.
    P-values less than 0.001 will be displayed as '<.001'.

    Args:
        df (pd.DataFrame): The input DataFrame.
        caption (str): The caption for the LaTeX table.

    Returns:
        str: A string containing the LaTeX table code.
    """
    latex_code = []

    # --- LaTeX table setup ---
    latex_code.append(r"\begingroup % To limit the scope of \renewcommand{\arraystretch}")
    latex_code.append(r"\renewcommand{\arraystretch}{1.2} % Adjust this value to control row spacing (increased for readability)")
    latex_code.append("")
    latex_code.append(r"    \centering")
    latex_code.append(r"    \begingroup")
    latex_code.append(r"    \footnotesize") # Using footnotesize for a potentially wide table
    latex_code.append(r"    \setlength{\tabcolsep}{4pt}") # Adjust column separation if needed
    latex_code.append(r"    \setlength{\extrarowheight}{1pt}")

    # --- Define tabular environment columns ---
    # 1 for Item (p-column), 3 for percentages (S-columns), 3 sets of (LOR S, CI p, p S)
    # Total columns: 1 + 3 + (3 * 3) = 13 columns
    # Adjust p-column widths to accommodate content.
    latex_code.append(r"  \begin{tabular}{p{4.0cm} c c c c c c c c c c c c }")
    # --- Table Headers ---
    latex_code.append(r"        \toprule")
    # First header row (multi-column for comparisons)
    latex_code.append(r"        \textbf{Item} & {\textbf{Spiritual\%}} & {\textbf{Psychosis\%}} & {\textbf{Ayahuasca\%}} & \multicolumn{3}{c}{\textbf{Comparison 1}} & \multicolumn{3}{c}{\textbf{Comparison 2}} & \multicolumn{3}{c}{\textbf{Comparison 3}} \\")
    latex_code.append(r"        \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}") # Rules below multicolumns
    # Second header row (specific metrics)
    latex_code.append(r"        & & & & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} \\")
    latex_code.append(r"        \midrule")

    previous_category = None

    for idx, row in df.iterrows():
        current_category = row.get('Category')
        item_name = idx

        # --- Handle Category Header ---
        if pd.notna(current_category) and current_category != previous_category:
            if previous_category is not None: # Add vertical space before new category if not the first
                latex_code.append(r"        [1ex]")
            latex_code.append(r"        \textbf{" + str(current_category) + r"} & & & & & & & & & & & & \\") # Category row
            previous_category = current_category
            latex_code.append(r"        [1ex]") # Vertical space after category header

        # --- Format Data for Current Row ---
        # Item name (indented)
        item_display = r"\hspace{1mm} " + str(item_name) if pd.notna(item_name) else ""

        # Percentages
        spiritual_perc = f"{row['Spiritual%']:.1f}" if pd.notna(row.get('Spiritual%')) else '-'
        psychosis_perc = f"{row['Psychosis%']:.1f}" if pd.notna(row.get('Psychosis%')) else '-'
        ayahuasca_perc = f"{row['Ayahuasca%']:.1f}" if pd.notna(row.get('Ayahuasca%')) else '-'

        # Function to format LOR, CI, and p-value for each comparison
        def format_lor_ci_p(lor, ci_lower, ci_upper, p_val):
            lor_str = f"{lor:.3f}" if pd.notna(lor) else '-'
            
            # P-value formatting
            p_val_str = '-'
            if pd.notna(p_val):
                if p_val < 0.001:
                    p_val_str = '<.001'
                else:
                    p_val_str = f"{p_val:.3f}"
            
            # CI formatting
            ci_str = '-'
            significance_asterisk = ""
            if pd.notna(ci_lower) and pd.notna(ci_upper):
                ci_str = f"[{ci_lower:.3f}, {ci_upper:.3f}]"
                if pd.notna(p_val) and p_val < 0.05:
                    significance_asterisk = "*"
            
            return lor_str, f"{ci_str}{significance_asterisk}", p_val_str

        lor1_str, ci1_str, p1_str = format_lor_ci_p(
            row.get('1_LOR'), row.get('1_LOR_CI_Lower'), row.get('1_LOR_CI_Upper'), row.get('1_p_val')
        )
        lor2_str, ci2_str, p2_str = format_lor_ci_p(
            row.get('2_LOR'), row.get('2_LOR_CI_Lower'), row.get('2_LOR_CI_Upper'), row.get('2_p_val')
        )
        lor3_str, ci3_str, p3_str = format_lor_ci_p(
            row.get('3_LOR'), row.get('3_LOR_CI_Lower'), row.get('3_LOR_CI_Upper'), row.get('3_p_val')
        )

        # --- Assemble LaTeX row ---
        latex_row = (
            f"{item_display} & "
            f"{spiritual_perc} & "
            f"{psychosis_perc} & "
            f"{ayahuasca_perc} & "
            f"{lor1_str} & {ci1_str} & {p1_str} & "
            f"{lor2_str} & {ci2_str} & {p2_str} & "
            f"{lor3_str} & {ci3_str} & {p3_str} \\\\"
        )
        latex_code.append(latex_row)

    latex_code.append(r"        \bottomrule")
    latex_code.append(r"    \end{tabular}")
    latex_code.append(f"    \\caption{{\\textbf{{{caption}}}}}")
    latex_code.append(r"    \label{tab:lor_summary}") # Add a specific label
    latex_code.append(r"\end{table}")
    latex_code.append(r"\endgroup{}") # Close the initial \begingroup

    return "\n".join(latex_code)




In [39]:
#For overall codes
#compute the log odds for the spiritualists and vips as a sanity check
df_both[['1_LOR','1_LOR_CI_Lower', '1_LOR_CI_Upper', '1_p_val']] = df_both.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Psychosis%'), axis=1)

#for the ayahuasca data we have, compute it too
df_both[['2_LOR','2_LOR_CI_Lower', '2_LOR_CI_Upper', '2_p_val']] = df_both.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Ayahuasca%'), axis=1)

#for the ayahuasca data we have, compute it too
df_both[['3_LOR','3_LOR_CI_Lower', '3_LOR_CI_Upper', '3_p_val']] = df_both.apply(lambda row : calculate_lor_and_ci(row, 'Psychosis%', 'Ayahuasca%'), axis=1)

df_both

latex_output = df_to_complex_lor_latex_table(df_both, caption="Comparison of Groups based on Logistic Odds Ratios")
print(latex_output)

\begingroup % To limit the scope of \renewcommand{\arraystretch}
\renewcommand{\arraystretch}{1.2} % Adjust this value to control row spacing (increased for readability)

\begin{table}[h!] % Using [h!] to prioritize placement
    \centering
    \begingroup
    \footnotesize
    \setlength{\tabcolsep}{4pt}
    \setlength{\extrarowheight}{1pt}
    \begin{tabular}{p{4.0cm} S[table-format=3.1] S[table-format=3.1] S[table-format=3.1] S[table-format=-1.3] p{2.2cm} S[table-format=1.3] S[table-format=-1.3] p{2.2cm} S[table-format=1.3] S[table-format=-1.3] p{2.2cm} S[table-format=1.3]}
        \toprule
        \textbf{Item} & {\textbf{Spiritual\%}} & {\textbf{Psychosis\%}} & {\textbf{Ayahuasca\%}} & \multicolumn{3}{c}{\textbf{Comparison 1}} & \multicolumn{3}{c}{\textbf{Comparison 2}} & \multicolumn{3}{c}{\textbf{Comparison 3}} \\
        \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
        & & & & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\te

In [44]:
#For during codes
#compute the log odds for the spiritualists and vips as a sanity check
df_during[['1_LOR','1_LOR_CI_Lower', '1_LOR_CI_Upper', '1_p_val']] = df_during.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Psychosis%'), axis=1)

#for the ayahuasca data we have, compute it too
df_during[['2_LOR','2_LOR_CI_Lower', '2_LOR_CI_Upper', '2_p_val']] = df_during.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Ayahuasca%'), axis=1)

#for the ayahuasca data we have, compute it too
df_during[['3_LOR','3_LOR_CI_Lower', '3_LOR_CI_Upper', '3_p_val']] = df_during.apply(lambda row : calculate_lor_and_ci(row, 'Psychosis%', 'Ayahuasca%'), axis=1)

df_during
latex_output = df_to_complex_lor_latex_table(df_during, caption="Comparison of Groups based on Logistic Odds Ratios")
print(latex_output)

\begingroup % To limit the scope of \renewcommand{\arraystretch}
\renewcommand{\arraystretch}{1.2} % Adjust this value to control row spacing (increased for readability)

    \centering
    \begingroup
    \footnotesize
    \setlength{\tabcolsep}{4pt}
    \setlength{\extrarowheight}{1pt}
    \begin{tabular}{p{4.0cm} S[table-format=3.1] S[table-format=3.1] S[table-format=3.1] S[table-format=-1.3] p{2.2cm} S[table-format=1.3] S[table-format=-1.3] p{2.2cm} S[table-format=1.3] S[table-format=-1.3] p{2.2cm} S[table-format=1.3]}
        \toprule
        \textbf{Item} & {\textbf{Spiritual\%}} & {\textbf{Psychosis\%}} & {\textbf{Ayahuasca\%}} & \multicolumn{3}{c}{\textbf{Comparison 1}} & \multicolumn{3}{c}{\textbf{Comparison 2}} & \multicolumn{3}{c}{\textbf{Comparison 3}} \\
        \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
        & & & & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}}

In [46]:
#For overall codes
#compute the log odds for the spiritualists and vips as a sanity check
df_after[['1_LOR','1_LOR_CI_Lower', '1_LOR_CI_Upper', '1_p_val']] = df_after.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Psychosis%'), axis=1)

#for the ayahuasca data we have, compute it too
df_after[['2_LOR','2_LOR_CI_Lower', '2_LOR_CI_Upper', '2_p_val']] = df_after.apply(lambda row : calculate_lor_and_ci(row, 'Spiritual%', 'Ayahuasca%'), axis=1)

#for the ayahuasca data we have, compute it too
df_after[['3_LOR','3_LOR_CI_Lower', '3_LOR_CI_Upper', '3_p_val']] = df_after.apply(lambda row : calculate_lor_and_ci(row, 'Psychosis%', 'Ayahuasca%'), axis=1)

df_after
latex_output = df_to_complex_lor_latex_table(df_after, caption="Comparison of Groups based on Logistic Odds Ratios")
print(latex_output)

\begingroup % To limit the scope of \renewcommand{\arraystretch}
\renewcommand{\arraystretch}{1.2} % Adjust this value to control row spacing (increased for readability)

    \centering
    \begingroup
    \footnotesize
    \setlength{\tabcolsep}{4pt}
    \setlength{\extrarowheight}{1pt}
  \begin{tabular}{p{4.0cm} c c c c c c c c c c c c }
        \toprule
        \textbf{Item} & {\textbf{Spiritual\%}} & {\textbf{Psychosis\%}} & {\textbf{Ayahuasca\%}} & \multicolumn{3}{c}{\textbf{Comparison 1}} & \multicolumn{3}{c}{\textbf{Comparison 2}} & \multicolumn{3}{c}{\textbf{Comparison 3}} \\
        \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
        & & & & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} & {\textbf{LOR}} & {\textbf{95\% CI}} & {\textbf{\textit{p}}} \\
        \midrule
\hspace{1mm} Visual imagery & 100.0 & 5.0 & 57.1 & 6.705 & [3.628, 9.781]* & <.001 & 3.675 & [0.810, 6.541]* & 0.036 & -3

#### An odds ratio (OR) is statistically significant when its confidence interval (CI) does not include 1, indicating a difference in the odds of the event between the two groups being compared.

In [24]:
import pandas as pd

def filter_df_by_strict_zero_crossing(df: pd.DataFrame, col1_name: str, col2_name: str, p_value_col_name: str) -> pd.DataFrame:
    """
    Filters a DataFrame, keeping only rows where both values are strictly positive
    or both values are strictly negative. This ensures the interval does not
    include or cross zero.

    Args:
        df (pd.DataFrame): The input DataFrame.
        col1_name (str): The name of the first numerical column.
        col2_name (str): The name of the second numerical column.
        p_value_col_name (str): The name of the column containing the p-values.

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    if col1_name not in df.columns or col2_name not in df.columns:
        raise ValueError(f"One or both columns '{col1_name}' or '{col2_name}' not found in the DataFrame.")
    
    keep_condition = (df[p_value_col_name] < 0.05/2) # Add the p-value check here

    df_filtered = df[keep_condition]
    
    return df_filtered

## Comparing results

In [25]:
# pick params for comparison
df_1 = df_after #pick df (both, during or after)
df_2 = df_after
i_1  = 2 # pick comparison (1: spiri-vip, 2: spiri-aya, 3: vip-aya)
i_2  = 3
i_3 = 1

filtered_df1 = filter_df_by_strict_zero_crossing(df_1, f'{i_1}_LOR_CI_Lower', f'{i_2}_LOR_CI_Upper', f'{i_1}_p_val')
filtered_df2 = filter_df_by_strict_zero_crossing(df_2, f'{i_2}_LOR_CI_Lower', f'{i_2}_LOR_CI_Upper', f'{i_2}_p_val')
filtered_df3 = filter_df_by_strict_zero_crossing(df_2, f'{i_3}_LOR_CI_Lower', f'{i_3}_LOR_CI_Upper', f'{i_3}_p_val')

#common elements
print('common elements')
#print(list(set(list(filtered_df1.index)) & set(list(filtered_df2.index)) & set(list(filtered_df2.index))), '\n')
print(list(set(list(filtered_df1.index)) & set(list(filtered_df2.index))), '\n')
print('comp1: ', list(filtered_df1.index), '\n')
print('comp2: ', list(filtered_df2.index), '\n')
#print('comp3: ', list(filtered_df2.index), '\n')

# the df of interest
filtered_df1

common elements
['Auditory', 'Olfactory', 'Minimal personification', 'Complex personification', 'Visual', 'Externally located', 'Internally individualized', 'Positive on relationships', 'Egocentric', 'Agency w/o individuation'] 

comp1:  ['Auditory', 'Egocentric', 'Externally located', 'Visual', 'Olfactory', 'Dissociative', 'Tactile', 'Simple structure', 'Commanding', 'Commenting', 'Internally individualized', 'Minimal personification', 'Complex personification', 'Agency w/o individuation', 'Positive on relationships'] 

comp2:  ['Visual imagery', 'Auditory', 'Egocentric', 'Thought-like', 'Externally located', 'Multimodal', 'Visual', 'Olfactory', 'Nonverbal', 'Allocentric', 'Elicit positive emotions', 'Voice knowledge', 'Volitional', 'Positive/helpful', 'Ability to influence', 'Recognizable', 'Change in influence', 'Abusive/violent', 'Supernatural narrative', 'Internally individualized', 'Externally individualized', 'Minimal personification', 'Complex personification', 'Agency w/o indi

,Spiritual%,Psychosis%,Ayahuasca%,1_LOR,1_LOR_CI_Lower,1_LOR_CI_Upper,1_p_val,2_LOR,2_LOR_CI_Lower,2_LOR_CI_Upper,2_p_val,3_LOR,3_LOR_CI_Lower,3_LOR_CI_Upper,3_p_val
Code,,,,,,,,,,,,,,,
Auditory,84.6,92.5,7.14,-0.762140,-2.254,0.729,0.9497,3.981016,2.490,5.472,0.0000,4.653960,2.953,6.355,0.0000
Egocentric,80.8,70.0,7.14,0.539129,-0.609,1.688,1.0727,3.734883,2.294,5.176,0.0000,3.149283,1.786,4.513,0.0000
Externally located,73.1,80.0,28.57,-0.385662,-1.517,0.745,1.5118,1.897555,0.814,2.981,0.0018,2.305348,1.147,3.464,0.0003
Visual,60.0,75.0,14.29,-0.614366,-1.659,0.430,0.7466,2.121142,0.981,3.262,0.0008,2.817780,1.601,4.035,0.0000
Olfactory,50.0,37.5,0.00,0.497838,-0.486,1.482,0.9637,4.394449,1.505,7.284,0.0086,3.942464,1.048,6.837,0.0228
Dissociative,46.2,30.0,0.00,0.675755,-0.331,1.683,0.5655,4.246029,1.356,7.136,0.0119,3.616745,0.712,6.522,0.0440
Tactile,46.2,22.5,7.14,1.050276,0.002,2.098,0.1486,2.223158,0.892,3.554,0.0032,1.222955,-0.184,2.630,0.2651
Simple structure,69.2,40.0,14.29,1.173017,0.149,2.197,0.0744,2.446862,1.280,3.614,0.0001,1.217172,0.077,2.358,0.1094
Commanding,26.9,67.5,71.43,-1.667008,-2.732,-0.602,0.0064,-1.897555,-2.981,-0.814,0.0018,-0.164339,-1.224,0.896,2.2838


In [26]:
# used previous cell to compute the common codes during/after for each pair
vip_aya = ['Abusive/violent', 'Family narrative', 'Elicit negative emotions', 'Dissociative', 'Multimodal', 'Minimal personification', 'Self-stigma', 'Complex personification', 'Egocentric', 'Recognizable', 'Nonverbal', 'Allocentric', 'Voice knowledge', 'Olfactory', 'Visual imagery', 'Boundary', 'Externally individualized', 'Externally located', 'Supernatural narrative', 'Positive on relationships', 'Visual', 'Agency w/o individuation', 'Positive/helpful', 'Internally individualized', 'Thought-like', 'Elicit positive emotions', 'Auditory', 'Negative on relationships']
spiri_aya = ['Archetypal', 'Simple structure', 'Dissociative', 'Minimal personification', 'Self-stigma', 'Complex personification', 'Egocentric', 'Nonverbal', 'Allocentric', 'Tactile', 'Olfactory', 'Idiosyncratic narrative', 'Commenting', 'Externally located', 'Positive on relationships', 'Visual', 'Commanding', 'Agency w/o individuation', 'Volitional', 'Change in frequency', 'Internally individualized', 'Gustatory', 'Auditory', 'Negative on relationships']

# now we look at the codes that are sig across groups and contexts
vip_spiri_aya = list(set(vip_aya) & set(spiri_aya))
print(vip_spiri_aya)

['Auditory', 'Negative on relationships', 'Olfactory', 'Dissociative', 'Minimal personification', 'Nonverbal', 'Self-stigma', 'Allocentric', 'Complex personification', 'Externally located', 'Internally individualized', 'Agency w/o individuation', 'Positive on relationships', 'Egocentric', 'Visual']


## Results Visualization


In [27]:
# let's use a forest plot!!
#!pip install forestplot

In [28]:
import forestplot as fp

### Reshape the dataframes

In [29]:
### 'Both' dataframe, containing the codes that are not exclusive to during or after the ayahuasca experience
df_during['Code'] = df_during.index

In [30]:
# --- 2. Identify columns to melt ---
id_vars = ['Code', 'Spiritual%', 'Psychosis%', 'Ayahuasca%']
value_vars = [col for col in df_both.columns if re.match(r'^\d_LOR(?:_CI_(?:Lower|Upper))?$', col)]

# --- 3. Melt the DataFrame to long format ---
df_long = pd.melt(df_both,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name='Metric',
                  value_name='Value')

# --- 4. Extract comparison number and value type from 'Metric' column ---
def parse_metric(metric_str):
    match = re.match(r'(\d)_(LOR(?:_CI_(?:Lower|Upper))?)', metric_str)
    if match:
        return match.groups()
    return None, None

df_long[['Comparison', 'ValueType']] = df_long['Metric'].apply(lambda x: pd.Series(parse_metric(x)))

# --- 5. Pivot the DataFrame back to get 'estimate', 'll', 'hl' columns ---
df_reshaped = df_long.pivot_table(index=['Code', 'Comparison'] + id_vars[1:],
                                  columns='ValueType',
                                  values='Value').reset_index()

# --- 6. Rename columns for forestplot compatibility ---
df_reshaped = df_reshaped.rename(columns={
    'LOR': 'estimate',
    'LOR_CI_Lower': 'll',
    'LOR_CI_Upper': 'hl'
})

# Ensure numeric types
df_reshaped['estimate'] = pd.to_numeric(df_reshaped['estimate'])
df_reshaped['ll'] = pd.to_numeric(df_reshaped['ll'])
df_reshaped['hl'] = pd.to_numeric(df_reshaped['hl'])

# Convert 'Comparison' to a categorical type and sort for consistent plotting order
df_reshaped['Comparison'] = pd.Categorical(df_reshaped['Comparison'], categories=['1', '2', '3'], ordered=True)
df_reshaped = df_reshaped.sort_values(by=['Code', 'Comparison']).reset_index(drop=True)

# --- 7. Prepare labels for the groupbylabel and display columns ---
modellabels_map = {
    '1': 'Comparison 1',
    '2': 'Comparison 2',
    '3': 'Comparison 3'
}
df_reshaped['Comparison_Label'] = df_reshaped['Comparison'].map(modellabels_map)

# Create new columns for display that will be conditionally filled
df_reshaped['display_varlabel'] = ''
df_reshaped['display_Spiritual%'] = ''
df_reshaped['display_Psychosis%'] = ''
df_reshaped['display_Ayahuasca%'] = ''

# Iterate through the DataFrame to fill the display columns
current_code = None
for index, row in df_reshaped.iterrows():
    if row['Code'] != current_code:
        # This is the first row for a new 'Code' group
        df_reshaped.loc[index, 'display_Spiritual%'] = row['Spiritual%']
        df_reshaped.loc[index, 'display_Psychosis%'] = row['Psychosis%']
        df_reshaped.loc[index, 'display_Ayahuasca%'] = row['Ayahuasca%']
        current_code = row['Code']

KeyError: "The following 'id_vars' are not present in the DataFrame: ['Code']"

In [ ]:
## the during df
# --- 2. Identify columns to melt ---
id_vars = ['Code', 'Spiritual%', 'Psychosis%', 'Ayahuasca%']
value_vars = [col for col in df_during.columns if re.match(r'^\d_LOR(?:_CI_(?:Lower|Upper))?$', col)]

# --- 3. Melt the DataFrame to long format ---
df_long = pd.melt(df_during,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name='Metric',
                  value_name='Value')

# --- 4. Extract comparison number and value type from 'Metric' column ---
def parse_metric(metric_str):
    match = re.match(r'(\d)_(LOR(?:_CI_(?:Lower|Upper))?)', metric_str)
    if match:
        return match.groups()
    return None, None

df_long[['Comparison', 'ValueType']] = df_long['Metric'].apply(lambda x: pd.Series(parse_metric(x)))

# --- 5. Pivot the DataFrame back to get 'estimate', 'll', 'hl' columns ---
df_reshaped = df_long.pivot_table(index=['Code', 'Comparison'] + id_vars[1:],
                                  columns='ValueType',
                                  values='Value').reset_index()

# --- 6. Rename columns for forestplot compatibility ---
df_reshaped = df_reshaped.rename(columns={
    'LOR': 'estimate',
    'LOR_CI_Lower': 'll',
    'LOR_CI_Upper': 'hl'
})

# Ensure numeric types
df_reshaped['estimate'] = pd.to_numeric(df_reshaped['estimate'])
df_reshaped['ll'] = pd.to_numeric(df_reshaped['ll'])
df_reshaped['hl'] = pd.to_numeric(df_reshaped['hl'])

# Convert 'Comparison' to a categorical type and sort for consistent plotting order
df_reshaped['Comparison'] = pd.Categorical(df_reshaped['Comparison'], categories=['1', '2', '3'], ordered=True)
df_reshaped = df_reshaped.sort_values(by=['Code', 'Comparison']).reset_index(drop=True)

# --- 7. Prepare labels for the groupbylabel and display columns ---
modellabels_map = {
    '1': 'Comparison 1',
    '2': 'Comparison 2',
    '3': 'Comparison 3'
}
df_reshaped['Comparison_Label'] = df_reshaped['Comparison'].map(modellabels_map)

# Create new columns for display that will be conditionally filled
df_reshaped['display_varlabel'] = ''
df_reshaped['display_Spiritual%'] = ''
df_reshaped['display_Psychosis%'] = ''
df_reshaped['display_Ayahuasca%'] = ''

# Iterate through the DataFrame to fill the display columns
current_code = None
for index, row in df_reshaped.iterrows():
    if row['Code'] != current_code:
        # This is the first row for a new 'Code' group
        df_reshaped.loc[index, 'display_Spiritual%'] = row['Spiritual%']
        df_reshaped.loc[index, 'display_Psychosis%'] = row['Psychosis%']
        df_reshaped.loc[index, 'display_Ayahuasca%'] = row['Ayahuasca%']
        current_code = row['Code']
df_reshaped_during = df_reshaped

## Plots!!!

In [ ]:
# --- 8. Plot the reshaped DataFrame using fp.forestplot with custom display columns ---
fp.forestplot(
    dataframe=df_reshaped,
    estimate="estimate",
    ll="ll",
    hl="hl",
    varlabel="Comparison_Label", # Use the new column with conditional empty strings
    capitalize= "capitalize",
    color_alt_rows=True,
    table=True, # Display a table on the right side
    rightannote=["display_Spiritual%", "display_Psychosis%", "display_Ayahuasca%"],
    right_annoteheaders=["Spiritual%", "Psychosis%", "Ayahuasca%"],
    xlabel="Log Odds Ratio (95% CI)",
    xticks=[-8, -4, 0, 4, 8],
    groupvar= 'Code',
    group_order = list(df_reshaped['Code'].unique()),
    # Adjusting offset to 0 for better alignment
    **{
        "markersize": 30,
        "offset": 0.0, # Setting offset to 0.0 to minimize misalignment
        "xlinestyle": (0, (10, 5)),
        "xlinecolor": ".8",
        "figsize": (10, 8),
    },
)

In [ ]:
# --- 8. Plot the reshaped DataFrame using fp.forestplot with custom display columns ---
fp.forestplot(
    dataframe=df_reshaped_during,
    estimate="estimate",
    ll="ll",
    hl="hl",
    varlabel="Comparison_Label", # Use the new column with conditional empty strings
    capitalize= "capitalize",
    color_alt_rows=True,
    table=True, # Display a table on the right side
    rightannote=["display_Spiritual%", "display_Psychosis%", "display_Ayahuasca%"],
    right_annoteheaders=["Spiritual%", "Psychosis%", "Ayahuasca%"],
    xlabel="Log Odds Ratio (95% CI)",
    xticks=[-8, -4, 0, 4, 8],
    groupvar= 'Code',
    group_order = list(df_reshaped_during['Code'].unique()),
    # Adjusting offset to 0 for better alignment
        # Adjusting offset to 0 for better alignment
    **{
        "markersize": 30,
        "offset": 0.0, # Setting offset to 0.0 to minimize misalignment
        "xlinestyle": (0, (10, 5)),
        "xlinecolor": ".8",
        "figsize": (8,40),
    },

)

## Stat test during/after


In [47]:
import scipy.stats as sc
from scipy.stats import fisher_exact, mannwhitneyu
from scipy.stats.contingency import chi2_contingency
from statsmodels.stats.contingency_tables import mcnemar
import statsmodels.api as sm

In [48]:
df = df_final.drop(['Question_text', 'Percentages', 'Total'], axis = 1).transpose()
df = df.drop(3)
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,B1,...,T7,T8,T9,T10,T11,T12,T13,T14,X1,X2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1,1
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1,1
5,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0,1
8,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
11,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1


In [76]:
def df_to_custom_latex_table(df, caption="Descriptive statistics of the voice phenomenology during and after ayahuasca."):
    """
    Converts a pandas DataFrame into a custom LaTeX table format,
    mimicking the provided structure with grouped rows and specific columns.
    NaN values in data columns will be represented by a hyphen '-'.

    Args:
        df (pd.DataFrame): The input DataFrame. Expected columns:
                           'Question 1', 'Question 2', 'test_name', 'statistic',
                           'P-value', 'Pcorr', 'dof'.
        caption (str): The caption for the LaTeX table.
        wrap_table_pos (str): Position for wraptable ('l' for left, 'r' for right).
        wrap_table_width (str): Width for wraptable (e.g., '7cm', '0.5\\textwidth').

    Returns:
        str: A string containing the LaTeX table code.
    """
    latex_code = []

    # Start of the wraptable environment
    latex_code.append(f"\\begin{{table}}")
    latex_code.append("    \\centering")
    latex_code.append("    \\begingroup")
    latex_code.append("    \\footnotesize")
    latex_code.append("    \\setlength{\\extrarowheight}{1pt}")

    # Define columns: 1st column ragged right, rest centered
    latex_code.append("    \\begin{tabular}{p{4cm} cccc}") # 1 custom p-column + 5 c-columns (6 total)

    latex_code.append("    \\toprule")
    # Header row
    latex_code.append("    \\textbf{Item} & \\textbf{Test name} & \\textbf{stat} & \\textbf{$p-value$} & \\textbf{$p_{corr}$} \\\\")
    latex_code.append("    \\midrule")

    previous_q1 = None
    for idx, row in df.iterrows():
        current_q1 = row['Question 1']
        current_q2 = row['Question 2']

        # Format data fields, handling NaN values with '-'
        test_name_val = '-' if pd.isnull(row['test_name']) else str(row['test_name'])
        statistic_val = '-' if pd.isnull(row['statistic']) else f"{row['statistic']:.2f}"
        p_value_val = '-' if pd.isnull(row['P-value']) else f"{row['P-value']:.3f}"
        pcorr_val = '-' if pd.isnull(row['Pcorr']) else f"{row['Pcorr']:.3f}" 

        # If Question 1 changes, print the bolded Question 1 header (and add spacing for previous group)
        if current_q1 != previous_q1:
            if previous_q1 is not None: # Add spacing after a group if it's not the very first group
                latex_code.append("    [1ex]")

            # If Q2 is empty/NaN for the current Q1, it means Q1 itself is the item with data (e.g., "Audible")
            # If Q2 has values, Q1 is just a category header (e.g., "Location")
            if pd.isnull(current_q2) or current_q2 == '':
                # This row is for a main item that also has data
                item_display = f"\\textbf{{{current_q1}}}"
            else:
                # This row is for a sub-item group, so first print Q1 as a category header
                item_display = f"\\hspace{{3mm}} {current_q2}" # Now set item_display for the sub-item
            previous_q1 = current_q1 # Update previous_q1 after processing the category header
        else:
            # If Question 1 is the same as the previous row, it's a sub-item
            item_display = f"\\hspace{{3mm}} {current_q2}"


        latex_code.append(
            f"    {item_display} & "
            f"{test_name_val} & "
            f"{statistic_val} & "
            f"{p_value_val} & "
            f"{pcorr_val}  \\\\"
        )
    latex_code.append("    [1ex]") # Add a final [1ex] after the last row

    latex_code.append("    \\bottomrule")
    latex_code.append("    \\end{tabular}")
    latex_code.append(f"    \\caption{{\\textbf{{{caption}}}}}")
    latex_code.append("    \\endgroup")
    latex_code.append("\\end{table}")

    return "\n".join(latex_code)

In [50]:
def run_mcnemar_test(contingency_table):
    """
    Performs McNemar's test on two paired categorical columns (before and after responses). Works on dichotomous data

    Parameters:
    df (pd.DataFrame): The dataset.
    col_before (str or int): The column name or index representing the "before" responses.
    col_after (str or int): The column name or index representing the "after" responses.

    Returns:
    float: The p-value from McNemar's test.
    """

    # Convert to numpy array for McNemar test
    table = contingency_table.values

    # Run McNemar's test
    result = mcnemar(table, exact=True) #what about third param

    return result.pvalue, result.statistic



def stat_test(df, col1, col2, paired=True):
    """
    Performs appropriate statistical test (McNemar's or Fisher's exact).

    Args:
        df: Pandas DataFrame.
        col1: Name of the first column (e.g., 'Before', 'ConditionA').
        col2: Name of the second column (e.g., 'After', 'ConditionB').
        paired: True if the data is paired (attempts McNemar's), False otherwise (attempts Fisher's).

    Returns:
        A dictionary containing the test results:
            'test_name': Name of the test performed.
            'p_value': P-value of the test.
            'statistic': Test statistic.
            'warning': A warning message if applicable.
    """
    results = {
        'test_name': None,
        'p_value': None,
        'statistic': None,
        'warning': None
    }

    # Basic input validation
    if col1 not in df.columns or col2 not in df.columns:
        results['warning'] = "One or both specified columns not found in the DataFrame."
        return results

    # Create the 2x2 contingency table directly from the 0/1 columns
    # pd.crosstab will correctly handle 0s and 1s as categories and create the 2x2 table.
    contingency_table = pd.crosstab(df[col1], df[col2])

    # Ensure the table is 2x2 and fill missing combinations with 0
    # This step is still important because if one combination doesn't exist, crosstab won't include it.
    # We explicitly define the index and columns for a 2x2 table of 0s and 1s.
    required_index = [0, 1]
    required_columns = [0, 1]
    
    # Reindex the contingency table to ensure it's a 2x2 matrix with 0s for missing cells
    ct_array = contingency_table.reindex(index=required_index, columns=required_columns, fill_value=0).values

    if paired:
        # --- Attempt McNemar's Test ---
        results['test_name'] = "McNemar's Test"
        b = ct_array[0, 1]  # Discordant: col1=0, col2=1
        c = ct_array[1, 0]  # Discordant: col1=1, col2=0

        # McNemar's test is based on discordant pairs. If there are no discordant pairs,
        # the p-value is typically 1, meaning no difference, or the test might be undefined.
        # It's common to fall back to Fisher's exact if discordant cells are zero.
        if b == 0 and c == 0:
            results['warning'] = "McNemar's test has zero discordant cells. Performing Fisher's Exact Test instead."
            try:
                statistic, p_value = fisher_exact(ct_array)
                results['test_name'] = "Fisher's Exact Test (Fallback)"
                results['statistic'] = statistic
                results['p_value'] = p_value
            except Exception as e:
                results['warning'] = f"Fisher's Exact Test (fallback) failed: {e}"
        else:
            try:
                mcnemar_result = mcnemar(ct_array, exact=True)
                results['statistic'] = mcnemar_result.statistic
                results['p_value'] = mcnemar_result.pvalue
            except Exception as e:
                results['warning'] = f"McNemar's test failed: {e}. Attempting Fisher's Exact Test as fallback."
                try:
                    statistic, p_value = fisher_exact(ct_array)
                    results['test_name'] = "Fisher's Exact Test (Fallback)"
                    results['statistic'] = statistic
                    results['p_value'] = p_value
                except Exception as fe:
                    results['warning'] += f" Fisher's Exact Test (fallback) also failed: {fe}"
    else:
        # --- Attempt Fisher's Exact Test directly (for unpaired 2x2) ---
        results['test_name'] = "Fisher's Exact Test"
        try:
            statistic, p_value = fisher_exact(ct_array)
            results['statistic'] = statistic
            results['p_value'] = p_value
        except Exception as e:
            results['warning'] = f"Fisher's Exact Test failed: {e}. Check your data format or if a 2x2 table can be formed."
            
    return results

In [51]:
contingency_table = pd.crosstab(df['O2'], df['G2'])
print(mcnemar(contingency_table))

pvalue      0.0625
statistic   0.0


In [52]:
sum(df['X1'])/14
#sum(df['X2'])/14

0.42857142857142855

In [53]:
#check if ability to control is sig
results = stat_test(df, 'X1',  'X2', paired= True)
results['p_value']

0.0625

In [69]:
df_final.loc[col1].Question_text 

'Tactile hallucination'

In [77]:
#for the modatlities table
col2_list = list(df_final['J1':'J9'].index)
results_list = []

for i, col1 in enumerate(list(df_final['B1':'B9'].index)):
    col2 = col2_list[i]

    #perform the test
    results = stat_test(df, col1,  col2, paired= True)
    p = results['p_value']

    #compute the corrected p value
    pcorr = p
    
    if pcorr is not None and pcorr < 0.05:
        sig = 'SIGNIFICANT'
    else:
        sig = 'not sig'
        

    # Append the results as a dictionary to the list
    results_list.append({
        'Question 1': df_final.loc[col1].Question_text ,  # Store column names, not indices
        'Question 2': df_final.loc[col2].Question_text ,
        'P-value': p,
        'Pcorr': pcorr,
        'Significance': sig,
        **results #unpack the dictionary
    })

# Create the DataFrame *outside* the loop
results_df = pd.DataFrame(results_list)
results_df.head()

print(df_to_custom_latex_table(results_df)) #printing latex table for report

\begin{table}
    \centering
    \begingroup
    \footnotesize
    \setlength{\extrarowheight}{1pt}
    \begin{tabular}{p{4cm} cccc}
    \toprule
    \textbf{Item} & \textbf{Test name} & \textbf{stat} & \textbf{$p-value$} & \textbf{$p_{corr}$} \\
    \midrule
    \hspace{3mm} ‘Auditory’ voices & McNemar's Test & 0.00 & 0.062 & 0.062  \\
    [1ex]
    \hspace{3mm} Olfactory hallucination & McNemar's Test & 0.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Gustatory hallucination & Fisher's Exact Test (Fallback) & - & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Dissociative experiences & McNemar's Test & 0.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Bodily states & McNemar's Test & 1.00 & 0.375 & 0.375  \\
    [1ex]
    \hspace{3mm} ‘Thought-like’ voices & Fisher's Exact Test (Fallback) & - & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Telepathic voices & McNemar's Test & 0.00 & 0.250 & 0.250  \\
    [1ex]
    \hspace{3mm} Multisensory voices & McNemar's Test & 1.00 & 0.625 & 0

In [78]:
col2_list = list(df_final['C1':'M4'].index)
results_list = []

for i, col1 in enumerate(list(df_final['K1':'M4'].index)):
    col2 = col2_list[i]

    #perform the test
    results = stat_test(df, col1,  col2, paired= True)
    p = results['p_value']

    #compute the corrected p value
    pcorr = p
    
    if pcorr is not None and pcorr < 0.05:
        sig = 'SIGNIFICANT'
    else:
        sig = 'not sig'
        

    # Append the results as a dictionary to the list
    results_list.append({
        'Question 1': df_final.loc[col1].Question_text ,  # Store column names, not indices
        'Question 2': df_final.loc[col2].Question_text ,
        'P-value': p,
        'Pcorr': pcorr,
        'Significance': sig,
        **results #unpack the dictionary
    })

# Create the DataFrame *outside* the loop
results_df = pd.DataFrame(results_list)
results_df
print(df_to_custom_latex_table(results_df)) #printing latex table for report

\begin{table}
    \centering
    \begingroup
    \footnotesize
    \setlength{\extrarowheight}{1pt}
    \begin{tabular}{p{4cm} cccc}
    \toprule
    \textbf{Item} & \textbf{Test name} & \textbf{stat} & \textbf{$p-value$} & \textbf{$p_{corr}$} \\
    \midrule
    \hspace{3mm} Internally located voices? & McNemar's Test & 1.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Externally located voices? & McNemar's Test & 1.00 & 0.625 & 0.625  \\
    [1ex]
    \hspace{3mm} ‘Boundary’ voices & Fisher's Exact Test (Fallback) & - & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} ‘Egocentric voices & McNemar's Test & 0.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} ‘Allocentric’ voices & Fisher's Exact Test (Fallback) & - & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Predominant & specific location reported? & McNemar's Test & 2.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Recurring voices & McNemar's Test & 1.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Simple structure & McNemar's 

In [79]:
col2_list = list(df_final['F1':'G9'].index)
results_list = []

for i, col1 in enumerate(list(df_final['N1':'O9'].index)):
    col2 = col2_list[i]

    #perform the test
    results = stat_test(df, col1,  col2, paired= True)
    p = results['p_value']

    #compute the corrected p value
    pcorr = p
    
    if pcorr is not None and pcorr < 0.05:
        sig = 'SIGNIFICANT'
    else:
        sig = 'not sig'
        

    # Append the results as a dictionary to the list
    results_list.append({
        'Question 1': df_final.loc[col1].Question_text ,  # Store column names, not indices
        'Question 2': df_final.loc[col2].Question_text ,
        'P-value': p,
        'Pcorr': pcorr,
        'Significance': sig,
        **results #unpack the dictionary
    })

# Create the DataFrame *outside* the loop
results_df = pd.DataFrame(results_list)
results_df
print(df_to_custom_latex_table(results_df))

\begin{table}
    \centering
    \begingroup
    \footnotesize
    \setlength{\extrarowheight}{1pt}
    \begin{tabular}{p{4cm} cccc}
    \toprule
    \textbf{Item} & \textbf{Test name} & \textbf{stat} & \textbf{$p-value$} & \textbf{$p_{corr}$} \\
    \midrule
    \hspace{3mm} Presence of voices & McNemar's Test & 3.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Tactile contacts & McNemar's Test & 0.00 & 0.500 & 0.500  \\
    [1ex]
    \hspace{3mm} State of mind & McNemar's Test & 1.00 & 0.625 & 0.625  \\
    [1ex]
    \hspace{3mm} Calm, relaxed state & McNemar's Test & 0.00 & 0.250 & 0.250  \\
    [1ex]
    \hspace{3mm} Anxious, stressed or alert state & McNemar's Test & 0.00 & 0.250 & 0.250  \\
    [1ex]
    \hspace{3mm} Depressed state & Fisher's Exact Test (Fallback) & - & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Events that recently happened & McNemar's Test & 0.00 & 1.000 & 1.000  \\
    [1ex]
    \hspace{3mm} Intensity of voices & McNemar's Test & 2.00 & 0.688 & 0.688  \\


# Add inter-rater k


In [ ]:
#import barbara's codings
docx_dir2 = "C:/Users/anaho/Desktop/research/Ayahuasca/DATA/codagesBarb"

# 1. Get all DOCX files in the directory, sorted alphanumerically
docx_files = sorted([f for f in os.listdir(docx_dir2) if f.endswith(".docx")])

all_data = {}  # Store data from all files, keyed by participant ID
df = pd.DataFrame()

#make question dict
question_dict = {}

 # 2. Iterate through each DOCX file
for docx_file in docx_files:
    docx_path = os.path.join(docx_dir2, docx_file)
    try:
        document = docx.Document(docx_path)
        num_tables = len(document.tables)
        print(f"Processing {num_tables} tables in {docx_file}")
        if num_tables == 0:
            print(f"No tables found in {docx_file}, skipping.")
            continue

    except docx.exceptions.EmptyPackageError as e:
        print(f"Error reading {docx_file}: {e}.  Skipping this file.")
        continue

    # Extract participant name from filename (remove ".docx")
    participant_name = os.path.splitext(docx_file)[0]  # e.g., "codage_01"

    # Extract participant ID (e.g., "01")
    participant_id = participant_name[-2:]
    
    all_table_data= [] #instantiate list

    # 3. Iterate through each table in the document
    for table_index, table in enumerate(document.tables):
        print(f"  Processing table {table_index + 1} of {num_tables} in {docx_file}")
        table_data = {}  # Store data for the current table, keyed by question code
        # 4. Extract data from the table, handling LetterNumber rows
        for row in table.rows:
            cells = [cell.text.strip() for cell in row.cells]  # Remove extra whitespace
            if cells:
                first_cell_value = cells[0]
                if re.match(r"^[A-Za-z]\d+$", first_cell_value) or re.match(r"^[A-Za-z]\d[A-Za-z]+$", first_cell_value) or re.match(r"^[A-Za-z]\d\d[A-Za-z]+$", first_cell_value):
                    question_code = first_cell_value
                    question_text = cells[1]  # Extract the question text
                    yes_val = cells[2].lower()
                    no_val = cells[3].lower()
                    note = cells[4]
                    response = None # Initialize response

                    if "x" in yes_val:
                        response = 1.0
                    elif "x" in no_val:
                        response = 0.0
                    else:
                         response = 0.0 #replacing by 0 the missing x

                    table_data[question_code] = {"response": response, "note": note}
                    question_dict[question_code] = question_text #store the question and text.

        
            '''    else:
                    print(
                        f"    Skipping row in {docx_file}, table {table_index + 1} because the first cell '{first_cell_value}' does not match 'A1' format.")
            else:
                print(f"    Skipping empty row in {docx_file}, table {table_index + 1}.")'''

        # 5. Store data for this participant and question
        all_table_data.append(table_data)  # Use participant_id

    # Flatten the list of dictionaries into a single dictionary for the participant
    all_table_data_flat = {}
    for item in all_table_data:
        all_table_data_flat.update(item)

    all_data[participant_id] = all_table_data_flat

In [ ]:
 # 6. Prepare data for Excel, transposing and combining
# Prepare data for Excel sheet
excel_data = []
# Get all unique questions
unique_questions = set()
for data in all_data.values():
    unique_questions.update(data.keys())
unique_questions = sorted(list(unique_questions))

# Create the header row
header_row = ["Participants"]  # Changed Header
for question in unique_questions:
    header_row.extend([f"{question}"])
excel_data.append(header_row)

# Create participant rows
for participant_id in participant_ids:
    participant_data = all_data[participant_id]
    participant_row = [participant_id] # Start with participant ID

    for question in unique_questions:
        if question in participant_data:
            # If the question exists, append the response
            # participant_row.extend([participant_data[question]['response'], participant_data[question]['note']]) # if including notes
            participant_row.extend([participant_data[question]['response']]) # only codes, no notes
        else:
            # If the question is not in participant_data, append NaN
            # You might print here for debugging if a question is unexpectedly missing
            # print(f"Question '{question}' not found for participant '{participant_id}', appending NaN.")
            participant_row.append(np.nan) # Append NaN for missing data
    excel_data.append(participant_row)

# 7. Put everything in a dataframe!!
df = pd.DataFrame(excel_data, columns = header_row).drop([0])
df_final2 = df.transpose().drop(['Participants'])



In [ ]:
df_final2.loc['N1':]

In [ ]:
from sklearn.metrics import cohen_kappa_score
df1 = df_final
df2 = df_final2

# 1. Find common rows (questions)
common_rows = df1.index.intersection(df2.index)
print(f"\nCommon rows: {list(common_rows)}")

# Filter dataframes to include only common rows
df1_common_rows = df1.loc[common_rows]
df2_common_rows = df2.loc[common_rows]

# 2. Find common participants (columns)
common_participants = df1.columns.intersection(df2.columns)
print(f"Common participants: {list(common_participants)}")

# Ensure there are enough common participants to choose from
if len(common_participants) < 5:
    print(f"\nWarning: Not enough common participants ({len(common_participants)}) to select 5 random ones. Selecting all common participants.")
    participants_to_compare = common_participants
else:
    # 3. Select 5 random common participants
    participants_to_compare = np.random.choice(common_participants, 5, replace=False)

print(f"\nSelected participants for comparison: {list(participants_to_compare)}")

# 4. Compute Cohen's Kappa and perc agreement for each selected participant
kappa_results = {}
perc_agreement_results = {}

for participant in participants_to_compare:
    if participant in df1_common_rows.columns and participant in df2_common_rows.columns:
        # Extract the series for the current participant from both dataframes, using only common rows
        ratings_df1 = df1_common_rows[participant]
        ratings_df2 = df2_common_rows[participant]

        # Attempt to convert to numeric type. 'coerce' will turn unconvertible values into NaN
        ratings_df1_numeric = pd.to_numeric(ratings_df1, errors='coerce')
        ratings_df2_numeric = pd.to_numeric(ratings_df2, errors='coerce')

        # Combine the two series into a temporary DataFrame to easily drop NaNs row-wise
        combined_ratings = pd.DataFrame({'df1': ratings_df1_numeric, 'df2': ratings_df2_numeric})

        # Drop rows where either rating is NaN (this handles original NaNs AND coerced NaNs)
        combined_ratings_cleaned = combined_ratings.dropna()

        # Extract the cleaned series for kappa calculation
        y1_cleaned = combined_ratings_cleaned['df1']
        y2_cleaned = combined_ratings_cleaned['df2']

        # Check if there's enough data left after dropping NaNs
        if len(y1_cleaned) == 0:
            print(f"Skipping Participant {participant}: No common non-NaN ratings found after filtering and numeric conversion.")
            continue
        elif len(y1_cleaned) < 2: # At least two data points needed for kappa
            print(f"Skipping Participant {participant}: Less than 2 common non-NaN ratings found after filtering and numeric conversion.")
            continue

        # Crucial check: Ensure there are at least two *distinct* classes in the data after cleaning
        # for cohen_kappa_score to make sense. If all values are the same, it implies zero variance
        # which can cause issues or result in an undefined kappa.
        if len(y1_cleaned.unique()) < 2 or len(y2_cleaned.unique()) < 2:
            print(f"Skipping Participant {participant}: One or both datasets have less than 2 unique rating categories after cleaning. Kappa is undefined.")
            continue
        
        # --- Compute Percentage Agreement ---
        agreements = (y1_cleaned == y2_cleaned).sum()
        total_observations = len(y1_cleaned)
        percentage_agreement = (agreements / total_observations) * 100
        perc_agreement_results[participant] = percentage_agreement


        # Compute Cohen's Kappa
        try:
            kappa = cohen_kappa_score(y1_cleaned, y2_cleaned)
            kappa_results[participant] = kappa
        except ValueError as e:
            print(f"Error computing kappa for Participant {participant}: {e}")
            print(f"Problematic data for {participant}:")
            print(f"  y1_cleaned: {y1_cleaned.tolist()}")
            print(f"  y2_cleaned: {y2_cleaned.tolist()}")
            # This is where you might see the "unknown and binary targets" error again if
            # it's related to something subtle like float representation of integers, etc.
    else:
        print(f"Participant {participant} not found in both filtered dataframes (this should not happen if selected from common_participants).")

if not kappa_results and not perc_agreement_results:
    print("No agreement scores could be computed for the selected participants.")
else:
    for participant in participants_to_compare:
        kappa = kappa_results.get(participant, 'N/A')
        perc_agreement = perc_agreement_results.get(participant, 'N/A')
        print(f"Participant {participant}: Kappa={kappa:.4f} | Percent Agreement={perc_agreement:.2f}%")
